<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===================== GQR 3D TDSE — HC vs HCB, side-by-side (PNG + MP4) =====================
# - Two matched conditions: HC (funnel+field) and HCB (funnel+field+tilt)
# - 3D split-operator; slices: XY@z=0, XZ@y=0, YZ@x=0 + X/Y/Z maximum-intensity projections
# - Per-condition PNG frames + MP4
# - Composite side-by-side PNG frames + MP4
# - “Luxury” clarity: EMA autoscale, gamma, macroblock padding
# ============================================================================================

!pip -q install imageio-ffmpeg

import os, math, sys
import numpy as np
from dataclasses import dataclass
import imageio.v2 as iio
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
from PIL import Image, ImageDraw

# -------- GPU selection (CuPy if present) --------
USE_GPU = False
try:
    import cupy as cp
    _ = cp.zeros((1,))
    USE_GPU = True
    xp = cp
    print("[backend] CuPy (GPU)")
except Exception:
    xp = np
    print("[backend] NumPy (CPU)")

# -------- small utils --------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_host(a):
    if USE_GPU and isinstance(a, cp.ndarray):
        return cp.asnumpy(a)
    return np.asarray(a)

def pad_to_macroblock(img_np, m=16):
    """Pad (H,W,3) uint8 to multiples of 'm' for better codec compatibility."""
    H, W = img_np.shape[:2]
    H2 = ((H + m - 1)//m)*m
    W2 = ((W + m - 1)//m)*m
    if (H2, W2) == (H, W):
        return img_np
    out = np.zeros((H2, W2, 3), dtype=np.uint8)
    out[:H, :W] = img_np
    return out

# -------- simulation config (match across both conditions) --------
# Time & frames
T_FINAL      = 14.0      # total time
DT           = 0.06      # time step
SAVE_EVERY   = 2         # steps per saved frame (larger -> fewer frames)
FPS          = 15        # movie fps

# Domain / grid (matches earlier “Man2-like” geometry/orientation, +x propagation)
Nx, Ny, Nz = 160, 96, 96
Lx, Ly, Lz = 70.0, 35.0, 35.0

# Rendering / look
GAMMA        = 0.70      # Power-law gamma
EMA_DECAY    = 0.90      # EMA for vmax stability
CMAP         = "inferno" # consistent colormap
PANEL_DPI    = 180       # per-panel DPI

# Output folders
ROOT = "GQR3D_HC_vs_HCB"
FRAMES_HC   = os.path.join(ROOT, "HC_frames")
FRAMES_HCB  = os.path.join(ROOT, "HCB_frames")
FRAMES_COMP = os.path.join(ROOT, "composite_frames")
MP4_DIR     = os.path.join(ROOT, "mp4")
SNAP_DIR    = os.path.join(ROOT, "snapshots")
for d in (FRAMES_HC, FRAMES_HCB, FRAMES_COMP, MP4_DIR, SNAP_DIR): ensure_dir(d)

# -------- grid, k-space, initial packet --------
@dataclass
class Grid3D:
    Nx:int; Ny:int; Nz:int
    Lx:float; Ly:float; Lz:float

def build_grid(g: Grid3D):
    dx, dy, dz = g.Lx/g.Nx, g.Ly/g.Ny, g.Lz/g.Nz
    x = xp.linspace(-g.Lx/2, g.Lx/2 - dx, g.Nx)
    y = xp.linspace(-g.Ly/2, g.Ly/2 - dy, g.Ny)
    z = xp.linspace(-g.Lz/2, g.Lz/2 - dz, g.Nz)
    X, Y, Z = xp.meshgrid(x, y, z, indexing='xy')
    return (x,y,z,X,Y,Z,dx,dy,dz)

def kspace(g: Grid3D, dx,dy,dz):
    if USE_GPU:
        kx = 2*xp.pi*cp.fft.fftfreq(g.Nx, d=float(dx))
        ky = 2*xp.pi*cp.fft.fftfreq(g.Ny, d=float(dy))
        kz = 2*xp.pi*cp.fft.fftfreq(g.Nz, d=float(dz))
    else:
        kx = 2*xp.pi*xp.fft.fftfreq(g.Nx, d=float(dx))
        ky = 2*xp.pi*xp.fft.fftfreq(g.Ny, d=float(dy))
        kz = 2*xp.pi*xp.fft.fftfreq(g.Nz, d=float(dz))
    KX, KY, KZ = xp.meshgrid(kx, ky, kz, indexing='xy')
    return (kx,ky,kz,KX,KY,KZ)

# physics constants
HBAR = 1.0; MASS = 1.0

# baseline potential with soft edges + central x-barrier + product pocket
def baseline_V(X,Y,Z, Lx,Ly,Lz):
    barrier_center = 0.0
    barrier_width  = 2.0
    barrier_height = 0.30
    pocket_x_start = 10.0
    pocket_depth   = -15.0

    V = xp.zeros_like(X, dtype=xp.float32)
    barrier = (xp.abs(X - barrier_center) < (barrier_width/2))
    V += barrier_height * barrier
    V += pocket_depth * (X > pocket_x_start)

    edge_soft = 6.0
    V += 0.02*(xp.tanh((xp.abs(X)-(Lx/2-edge_soft))/2)+1.0)
    V += 0.02*(xp.tanh((xp.abs(Y)-(Ly/2-edge_soft))/2)+1.0)
    V += 0.02*(xp.tanh((xp.abs(Z)-(Lz/2-edge_soft))/2)+1.0)
    return V

# simple complex absorber (imag part) near boundaries
def absorber(X,Y,Z,Lx,Ly,Lz, margin=6.0, eta=2.0e-4):
    def _cap(coord, L):
        r = xp.maximum(xp.abs(coord)-(L/2 - margin),0)/margin
        return r**4
    CAP = -1j*eta*(_cap(X,Lx)+_cap(Y,Ly)+_cap(Z,Lz))
    return CAP

# initial packet (left -> right along +x)
def init_packet(X,Y,Z):
    x0,y0,z0 = -20.0, 0.0, 0.0
    kx0, ky0, kz0 = 1.6, 0.0, 0.0
    sigx,sigy,sigz = 3.5, 4.0, 4.0
    psi = xp.exp(-((X-x0)**2/(2*sigx**2) + (Y-y0)**2/(2*sigy**2) + (Z-z0)**2/(2*sigz**2))).astype(xp.complex64)
    psi *= xp.exp(1j*(kx0*X + ky0*Y + kz0*Z))
    return psi

# -------- factors: H (aperture), C (field lens), B (tilt) --------
def V_H(X,Y,Z):
    # funnel: gently focuses Y,Z within a window in X
    kappa_yz = 0.04
    xwin_w   = 6.0
    X_win    = xp.exp(-(X**2)/(2*xwin_w**2))
    return 0.5*kappa_yz*((Y**2)+(Z**2))*X_win

def V_C(X,Y,Z, t):
    # field lens, DC + small AC along +x (keeps “buzz” pushed forward)
    E_dc_C, E0_C, w_C = 0.08, 0.14, 0.10
    # use math.sin to avoid any backend ambiguity; t is scalar
    import math
    return -(E_dc_C + E0_C*math.sin(w_C*t))*X

def V_B(X,Y,Z):
    # tilt/prism: small bias in +Y to steer post-barrier direction
    B = 0.22  # tweakable; positive -> up; negative -> down
    return -B * Y

# condition builders (return total V for given t)
def V_HC_total(V0, CAP, X,Y,Z, t):
    return (V0 + CAP + V_H(X,Y,Z) + V_C(X,Y,Z, t)).astype(xp.complex64)

def V_HCB_total(V0, CAP, X,Y,Z, t):
    return (V0 + CAP + V_H(X,Y,Z) + V_C(X,Y,Z, t) + V_B(X,Y,Z)).astype(xp.complex64)

# -------- split operator setup --------
def kinetic_phase(KX,KY,KZ, dt):
    T_k = (HBAR**2)*(KX**2 + KY**2 + KZ**2)/(2*MASS)
    return xp.exp(-1j * T_k * dt / HBAR)

def step_tdse(psi, V, expT, dt):
    """One split-operator step with correct half-step potential factors."""
    psi = psi * xp.exp(-1j*V* (dt/(2*HBAR)))
    psi_k = xp.fft.fftn(psi, axes=(0,1,2))
    psi_k *= expT
    psi = xp.fft.ifftn(psi_k, axes=(0,1,2))
    psi = psi * xp.exp(-1j*V* (dt/(2*HBAR)))
    return psi

# -------- panel rendering --------
def render_panels(dens_host, extents, vmax, tag_text):
    """
    dens_host: (Ny, Nx, Nz) on CPU
    extents: (extent_xy, extent_xz, extent_yz)
    returns PIL.Image RGB (uint8), tiled 2x3 with titles
    """
    Ny, Nx, Nz = dens_host.shape
    extent_xy, extent_xz, extent_yz = extents

    data_panels = [
        ("XY @ z=0",    dens_host[:, :, Nz//2], extent_xy),
        ("XZ @ y=0",    dens_host[Ny//2, :, :].T, extent_xz),
        ("YZ @ x=0",    dens_host[:, Nx//2, :].T, extent_yz),
        ("X-MIP (max)", dens_host.max(axis=2),   extent_xy),        # (Ny,Nx)
        ("Y-MIP (max)", dens_host.max(axis=0).T, extent_yz),        # (Nx,Nz)->(Nz,Nx)
        ("Z-MIP (max)", dens_host.max(axis=1),   extent_xz),        # (Ny,Nz)
    ]

    w_in, h_in = 4.2, 2.7  # per-panel size (inches)
    imgs = []
    vmax = max(vmax, 1e-12)
    for title, arr, extent in data_panels:
        fig, ax = plt.subplots(figsize=(w_in, h_in), dpi=PANEL_DPI)
        ax.set_xticks([]); ax.set_yticks([])
        im = ax.imshow(arr, origin='lower', extent=extent,
                       cmap=CMAP, norm=PowerNorm(gamma=GAMMA, vmin=0.0, vmax=vmax), aspect='auto')
        ax.set_title(title, fontsize=11)
        fig.canvas.draw()
        # prefer buffer_rgba to avoid FigureCanvasAgg tostring deprecation
        rgb = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[1],
                                                                              fig.canvas.get_width_height()[0], 4)[..., :3]
        plt.close(fig)
        imgs.append(Image.fromarray(rgb))

    # tile 2x3
    W, H = imgs[0].size
    gap = 8
    pad = 50
    out = Image.new("RGB", (2*W + gap + 2*pad, 3*H + 2*gap + 2*pad), (255,255,255))
    draw = ImageDraw.Draw(out)

    coords = [(pad, pad), (pad+W+gap, pad),
              (pad, pad+H+gap), (pad+W+gap, pad+H+gap),
              (pad, pad+2*H+2*gap), (pad+W+gap, pad+2*H+2*gap)]
    for img, (x0,y0) in zip(imgs, coords):
        out.paste(img, (x0,y0))

    # label
    draw.text((pad, 10), tag_text, fill=(0,0,0))

    # pad for codec
    out_np = pad_to_macroblock(np.array(out, dtype=np.uint8), 16)
    return Image.fromarray(out_np)

# -------- run one condition and save PNG frames + MP4 + snapshot --------
def run_condition(tag, V_total_func, out_frames_dir, out_mp4_path, snapshot_path):
    g = Grid3D(Nx=Nx, Ny=Ny, Nz=Nz, Lx=Lx, Ly=Ly, Lz=Lz)
    x,y,z,X,Y,Z,dx,dy,dz = build_grid(g)
    kx,ky,kz,KX,KY,KZ = kspace(g, dx,dy,dz)

    V0  = baseline_V(X,Y,Z, Lx,Ly,Lz).astype(xp.complex64)
    CAP = absorber(X,Y,Z, Lx,Ly,Lz).astype(xp.complex64)

    psi = init_packet(X,Y,Z)
    norm0 = xp.sqrt(xp.sum(xp.abs(psi)**2)*dx*dy*dz)
    psi /= norm0

    expT = kinetic_phase(KX,KY,KZ, DT)
    nsteps  = int(T_FINAL/DT)
    framesN = max(1, nsteps//SAVE_EVERY)

    # plotting extents
    extent_xy = (-Lx/2, Lx/2 - dx, -Ly/2, Ly/2 - dy)
    extent_xz = (-Lx/2, Lx/2 - dx, -Lz/2, Lz/2 - dz)
    extent_yz = (-Ly/2, Ly/2 - dy, -Lz/2, Lz/2 - dz)
    extents = (extent_xy, extent_xz, extent_yz)

    ema_vmax = 0.02
    png_paths = []
    writer = iio.get_writer(out_mp4_path, fps=FPS, codec="libx264", quality=9)

    print(f"[run] {tag}: steps={nsteps}, frames≈{framesN}")
    frame_counter = 0
    for s in range(nsteps):
        t = (s+1)*DT
        V = V_total_func(V0, CAP, X,Y,Z, t)
        psi = step_tdse(psi, V, expT, DT)

        if (s % SAVE_EVERY)==0 or s==nsteps-1:
            dens = xp.abs(psi)**2
            dens_host = to_host(dens)

            # robust EMA vmax over multiple views
            xy   = dens_host[:, :, Nz//2]
            xz   = dens_host[Ny//2, :, :].T
            yz   = dens_host[:, Nx//2, :].T
            mipx = dens_host.max(axis=1)       # (Ny,Nz)
            mipy = dens_host.max(axis=0).T     # (Nz,Nx)
            mipz = dens_host.max(axis=2)       # (Ny,Nx)

            vmax_now = np.max([
                np.percentile(xy,  99.5),
                np.percentile(xz,  99.5),
                np.percentile(yz,  99.5),
                np.percentile(mipx,99.5),
                np.percentile(mipy,99.5),
                np.percentile(mipz,99.5),
            ])
            ema_vmax = EMA_DECAY*ema_vmax + (1-EMA_DECAY)*max(vmax_now, 1e-12)

            tile = render_panels(dens_host, extents, ema_vmax, f"{tag}  t={t:.2f}")
            png_path = os.path.join(out_frames_dir, f"{tag}_{frame_counter:04d}.png")
            tile.save(png_path, optimize=True)
            png_paths.append(png_path)
            writer.append_data(np.array(tile))
            frame_counter += 1

    writer.close()
    print(f"[{tag}] wrote MP4:", out_mp4_path)

    # snapshot = last frame
    if png_paths:
        Image.open(png_paths[-1]).save(snapshot_path)
        print(f"[{tag}] snapshot:", snapshot_path)

    return png_paths

# -------- composite builder (side-by-side) --------
def build_composite(frames_A, frames_B, out_dir, out_mp4_path, label_A="HC", label_B="HCB"):
    ensure_dir(out_dir)
    n = min(len(frames_A), len(frames_B))
    writer = iio.get_writer(out_mp4_path, fps=FPS, codec="libx264", quality=9)
    comp_paths = []
    for i in range(n):
        imA = Image.open(frames_A[i]).convert("RGB")
        imB = Image.open(frames_B[i]).convert("RGB")
        # equalize height
        h = max(imA.height, imB.height)
        wA = int(imA.width * (h/imA.height))
        wB = int(imB.width * (h/imB.height))
        imA = imA.resize((wA, h), Image.BICUBIC)
        imB = imB.resize((wB, h), Image.BICUBIC)
        gap = 12
        out = Image.new("RGB", (wA + gap + wB, h), (255,255,255))
        out.paste(imA, (0,0))
        out.paste(imB, (wA+gap, 0))

        # annotate headers
        draw = ImageDraw.Draw(out)
        draw.rectangle([0,0, wA, 28], fill=(255,255,255))
        draw.rectangle([wA+gap,0, wA+gap+wB, 28], fill=(255,255,255))
        draw.text((8,6),  label_A, fill=(0,0,0))
        draw.text((wA+gap+8,6), label_B, fill=(0,0,0))

        out_np = pad_to_macroblock(np.array(out, dtype=np.uint8), 16)
        p = os.path.join(out_dir, f"HC_vs_HCB_{i:04d}.png")
        Image.fromarray(out_np).save(p, optimize=True)
        comp_paths.append(p)
        writer.append_data(out_np)
    writer.close()
    print("[composite] wrote MP4:", out_mp4_path)

    # composite snapshot (last)
    if comp_paths:
        snap = os.path.join(SNAP_DIR, "HC_vs_HCB_snapshot.png")
        Image.open(comp_paths[-1]).save(snap)
        print("[composite] snapshot:", snap)

    return comp_paths

# ================================= RUN BOTH CONDITIONS =================================
# HC
frames_HC = run_condition(
    tag="HC",
    V_total_func=V_HC_total,
    out_frames_dir=FRAMES_HC,
    out_mp4_path=os.path.join(MP4_DIR, "HC.mp4"),
    snapshot_path=os.path.join(SNAP_DIR, "HC_snapshot.png"),
)

# HCB
frames_HCB = run_condition(
    tag="HCB",
    V_total_func=V_HCB_total,
    out_frames_dir=FRAMES_HCB,
    out_mp4_path=os.path.join(MP4_DIR, "HCB.mp4"),
    snapshot_path=os.path.join(SNAP_DIR, "HCB_snapshot.png"),
)

# Composite (side-by-side)
comp_paths = build_composite(
    frames_A=frames_HC,
    frames_B=frames_HCB,
    out_dir=FRAMES_COMP,
    out_mp4_path=os.path.join(MP4_DIR, "HC_vs_HCB.mp4"),
    label_A="HC (H+C lens)", label_B="HCB (H+C+tilt)"
)

print("\n[Done]")
print(" Per-condition frames:", FRAMES_HC, FRAMES_HCB)
print(" Composite frames:", FRAMES_COMP)
print(" MP4s:", MP4_DIR)
print(" Snapshots:", SNAP_DIR)

[backend] CuPy (GPU)
[run] HC: steps=233, frames≈116
[HC] wrote MP4: GQR3D_HC_vs_HCB/mp4/HC.mp4
[HC] snapshot: GQR3D_HC_vs_HCB/snapshots/HC_snapshot.png
[run] HCB: steps=233, frames≈116
[HCB] wrote MP4: GQR3D_HC_vs_HCB/mp4/HCB.mp4
[HCB] snapshot: GQR3D_HC_vs_HCB/snapshots/HCB_snapshot.png
[composite] wrote MP4: GQR3D_HC_vs_HCB/mp4/HC_vs_HCB.mp4
[composite] snapshot: GQR3D_HC_vs_HCB/snapshots/HC_vs_HCB_snapshot.png

[Done]
 Per-condition frames: GQR3D_HC_vs_HCB/HC_frames GQR3D_HC_vs_HCB/HCB_frames
 Composite frames: GQR3D_HC_vs_HCB/composite_frames
 MP4s: GQR3D_HC_vs_HCB/mp4
 Snapshots: GQR3D_HC_vs_HCB/snapshots
